In [1]:
!pip install llama-index llama-index-experimental llama-index-llms-groq gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.1/186.1 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import textwrap
import zipfile
import os
import io
import shutil

import pandas as pd
import gradio as gr

from datetime import datetime
from google.colab import userdata
from llama_index.core import Settings
from llama_index.llms.groq import Groq
from llama_index.experimental.query_engine import PandasQueryEngine
from llama_index.core import PromptTemplate
from llama_index.experimental.query_engine.pandas import PandasInstructionParser
from llama_index.core.query_pipeline import (QueryPipeline as QP, Link, InputComponent)


In [6]:
api_key = userdata.get('GROQ_API')

# Configuração do modelo
llm = Groq(model="llama3-70b-8192", api_key=api_key)

# Pipeline de consulta
'''Função para obter uma descrição das colunas do DataFrame'''
def descricao_colunas(df):
    descricao = '\n'.join([f"`{col}`: {str(df[col].dtype)}" for col in df.columns])
    return "Aqui estão os detalhes das colunas do dataframe:\n" + descricao

'''Definição de módulos da pipeline'''
def pipeline_consulta(df):
    instruction_str = (
        "1. Converta a consulta para código Python executável usando Pandas.\n"
        "2. A linha final do código deve ser uma expressão Python que possa ser chamada com a função `eval()`.\n"
        "3. O código deve representar uma solução para a consulta.\n"
        "4. IMPRIMA APENAS A EXPRESSÃO.\n"
        "5. Não coloque a expressão entre aspas.\n"
        "6. analise o resultado formule uma resposta valida.\n"
        "7. execute até 4 vezes para validar o resultado.\n"
        "8. validar se o resultado contem na df original.\n"

        )

    pandas_prompt_str = (
        "Você está trabalhando com um dataframe do pandas em Python chamado `df`.\n"
        "{colunas_detalhes}\n\n"
        "Este é o resultado de `print(df.head())`:\n"
        "{df_str}\n\n"
        "Siga estas instruções:\n"
        "{instruction_str}\n"
        "Consulta: {query_str}\n\n"
        "Expressão:"
)

    response_synthesis_prompt_str = (
       "Dada uma pergunta de entrada, atue como analista de dados e elabore uma resposta a partir dos resultados da consulta. se necessario execute mais de uma vez\n"
       "Responda de forma natural, sem introduções como 'A resposta é:' ou algo semelhante.\n"
       "Consulta: {query_str}\n\n"
       "Instruções do Pandas (opcional):\n{pandas_instructions}\n\n"
       "Saída do Pandas: {pandas_output}\n\n"
       "Resposta: \n\n"
       "Ao final, exibir o código usado em para gerar a resposta, no formato: O código utilizado foi `{pandas_instructions}`"
    )

    pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str,
    df_str=df.head(5),
    colunas_detalhes=descricao_colunas(df)
)

    pandas_output_parser = PandasInstructionParser(df)
    response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

    '''Criação do Query Pipeline'''
    qp = QP(
        modules={
            "input": InputComponent(),
            "pandas_prompt": pandas_prompt,
            "llm1": llm,
            "pandas_output_parser": pandas_output_parser,
            "response_synthesis_prompt": response_synthesis_prompt,
            "llm2": llm,
        },
        verbose=True,
    )
    qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
    qp.add_links(
        [
            Link("input", "response_synthesis_prompt", dest_key="query_str"),
            Link("llm1", "response_synthesis_prompt", dest_key="pandas_instructions"),
            Link("pandas_output_parser", "response_synthesis_prompt", dest_key="pandas_output"),
        ]
    )
    qp.add_link("response_synthesis_prompt", "llm2")
    return qp

def extrair_zip_e_carregar_csvs_em_memoria(caminho_zip):
    """
    Extrai todos os arquivos CSV de um ZIP e os carrega em DataFrames Pandas em memória.
    Retorna um dicionário onde a chave é o nome do arquivo CSV (sem caminho) e o valor é o DataFrame.
    """
    if not zipfile.is_zipfile(caminho_zip):
        raise ValueError("O arquivo fornecido não é um arquivo ZIP válido.")

    dfs = {}
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        csv_files_in_zip = [name for name in zip_ref.namelist() if name.lower().endswith('.csv')]

        if not csv_files_in_zip:
            raise FileNotFoundError("Nenhum arquivo CSV encontrado dentro do arquivo ZIP.")

        for csv_file_name in csv_files_in_zip:
            try:
                with zip_ref.open(csv_file_name) as file:
                    # Lê o arquivo CSV diretamente da memória
                    df = pd.read_csv(io.TextIOWrapper(file, 'utf-8'))
                    # Usa apenas o nome do arquivo, sem o caminho completo dentro do zip
                    df_name = os.path.basename(csv_file_name)
                    dfs[df_name] = df
            except Exception as e:
                print(f"Erro ao ler o CSV '{csv_file_name}' do ZIP: {e}")
                # Opcional: ignorar arquivos CSV que não podem ser lidos ou levantar um erro mais específico

    if not dfs:
        raise ValueError("Nenhum arquivo CSV válido pôde ser carregado do ZIP.")

    return dfs

# Função para carregar os dados
def carregar_dados(caminho_arquivo, df_estado):
    if caminho_arquivo is None or caminho_arquivo == "":
        return "Por favor, faça o upload de um arquivo (CSV ou ZIP) para analisar.", pd.DataFrame(), df_estado

    try:
        if caminho_arquivo.lower().endswith('.zip'):
            dfs_dict = extrair_zip_e_carregar_csvs_em_memoria(caminho_arquivo)
            # Para exibição, mostramos a cabeça do primeiro DataFrame encontrado
            first_df_name = list(dfs_dict.keys())[0]

            df1_name = list(dfs_dict.keys())[0]
            df2_name = list(dfs_dict.keys())[1]
            df1 = dfs_dict[df1_name]
            df2 = dfs_dict[df2_name]
            # Aqui você faria uma junção, agregação ou comparação entre df1 e df2
            try:
                merged_df = pd.merge(df1, df2, on='CHAVE DE ACESSO', how='inner')
            except KeyError:
                response_content += "\nNão foi possível fazer o merge direto por 'coluna_comum'."
            return "Arquivo carregado com sucesso!", merged_df.head(), merged_df
            #(f"Arquivo ZIP descompactado e {len(dfs_dict)} CSV(s) carregados com sucesso! "
            #        f"Exibindo a cabeça do '{first_df_name}'.", dfs_dict[first_df_name].head(), dfs_dict)
        elif caminho_arquivo.lower().endswith('.csv'):
            df = pd.read_csv(caminho_arquivo)
            # Se for um único CSV, armazena em um dicionário para consistência
            return "Arquivo carregado com sucesso!", df.head(), df
        else:
            return "Formato de arquivo não suportado. Por favor, faça upload de um arquivo CSV ou ZIP.", pd.DataFrame(), df_estado
    except Exception as e:
        return f"Erro ao carregar arquivo: {str(e)}", pd.DataFrame(), df_estado

# Função para processar a pergunta
def processar_pergunta(pergunta, df_estado):
    if df_estado is not None and pergunta:
        qp = pipeline_consulta(df_estado)
        resposta = qp.run(query_str=pergunta)
        return resposta.message.content
    return ""

# Função para resetar a aplicação
def resetar_aplicação():
    return None, "A aplicação foi resetada. Por favor, faça upload de um novo arquivo CSV.", pd.DataFrame(), "", None, [], ""

# Criação da interface gradio
with gr.Blocks(theme="Soft") as app:

    # Título da app
    gr.Markdown("# Analisando os dados🔎🎲")

    # Descrição
    gr.Markdown("""
    Carregue um arquivo ZIP e faça perguntas sobre os dados. A cada pergunta, você poderá
    visualizar a resposta. Se você quiser analisar um novo dataset, basta clicar em "Quero analisar outro dataset" ao final da página.
    """)

    # Campo de entrada de arquivos
    input_arquivo = gr.File(file_count="single", type="filepath", label="Upload CSV", file_types=[".csv", ".zip"])

    # Status de upload
    upload_status = gr.Textbox(label="Status do Upload:")

    # Tabela de dados
    tabela_dados = gr.DataFrame()

    # Exemplos de perguntas
    gr.Markdown("""
    Exemplos de perguntas:
    1. Qual a origem que tem mais itens?
    2. Qual o valor total das notas fiscais ?
    3. Qual é a origem que mais faturou em notas fiscais?
    4. Qual a maior notas fiscais e que itens foram comprados?
    """)

    # Campo de entrada de texto
    input_pergunta = gr.Textbox(label="Digite sua pergunta sobre os dados")

    # Botão de envio posicionado após a pergunta
    botao_submeter = gr.Button("Enviar")

    # Componente de resposta
    output_resposta = gr.Textbox(label="Resposta")

    # Botão para resetar a aplicação
    botao_resetar = gr.Button("Quero analisar outro dataset!")

    # Gerenciamento de estados
    df_estado = gr.State(value=None)

    # Conectando funções aos componentes
    input_arquivo.change(fn=carregar_dados,
                         inputs=[input_arquivo, df_estado],
                         outputs=[upload_status, tabela_dados, df_estado])

    botao_submeter.click(fn=processar_pergunta,
                         inputs=[input_pergunta, df_estado],
                         outputs=output_resposta)

    botao_resetar.click(fn=resetar_aplicação,
                        inputs=[],
                        outputs=[input_arquivo, upload_status, tabela_dados, output_resposta, input_pergunta])

    app.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0e949e70330fc22db0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


> Running module input with input: 
query_str: qual o nome do destinatorios que mais recebeu produtos e serviços e liste 10 produtos e serviço valores totais.

> Running module pandas_prompt with input: 
query_str: qual o nome do destinatorios que mais recebeu produtos e serviços e liste 10 produtos e serviço valores totais.

> Running module llm1 with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df`.
Aqui estão os detalhes das colunas do dataframe:
`CHAVE DE ACESSO`: object
`MODELO_x`: object
`SÉRIE_x`: int64
`NÚMERO_x`: int6...

> Running module pandas_output_parser with input: 
input: assistant: Aqui está a expressão Python que resolve a consulta:
```
df.groupby('NOME DESTINATÁRIO_x')['VALOR TOTAL'].sum().sort_values(ascending=False).head(10).index.tolist() + df.groupby('NOME DESTI...

> Running module response_synthesis_prompt with input: 
query_str: qual o nome do destinatorios que mais recebeu produtos e serviços e liste 10 produtos e serv

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/llama_index/experimental/query_engine/pandas/output_parser.py", line 42, in default_output_processor
    tree = ast.parse(output)
           ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<unknown>", line 5
    df.groupby('CPF/CNPJ Emitente_x')['VALOR TOTAL'].sum().sort_values(ascending=False).head(10')
                                                                                               ^
SyntaxError: unterminated string literal (detected at line 5)


> Running module input with input: 
query_str: Qual é a distribuição dos valores totais das operações? Isso pode ajudar a identificar se há uma concentração de valores em uma determinada faixa.

> Running module pandas_prompt with input: 
query_str: Qual é a distribuição dos valores totais das operações? Isso pode ajudar a identificar se há uma concentração de valores em uma determinada faixa.

> Running module llm1 with input: 
messages: Você está trabalhando com um dataframe do pandas em Python chamado `df`.
Aqui estão os detalhes das colunas do dataframe:
`CHAVE DE ACESSO`: object
`MODELO_x`: object
`SÉRIE_x`: int64
`NÚMERO_x`: int6...

> Running module pandas_output_parser with input: 
input: assistant: df['VALOR TOTAL'].value_counts()

> Running module response_synthesis_prompt with input: 
query_str: Qual é a distribuição dos valores totais das operações? Isso pode ajudar a identificar se há uma concentração de valores em uma determinada faixa.
pandas_instructions: assistant: df[